```{contents}
```
## **Agent in LangGraph**

An **Agent in LangGraph** is a **stateful, autonomous decision-making component** implemented as a **cyclic graph** that repeatedly **observes → reasons → acts → learns from feedback** until a goal is achieved or a stopping condition is reached.

LangGraph does **not** treat an agent as a single function or class.
An agent is an **execution pattern over a graph**.

---

### **1. Why Agents Need LangGraph**

Traditional LangChain agents are **linear** and **stateless**.
Real agents must:

| Capability        | Why It Matters      |
| ----------------- | ------------------- |
| Persistent memory | Maintain context    |
| Iteration         | Improve solutions   |
| Tool feedback     | Ground reasoning    |
| Planning          | Solve complex tasks |
| Self-correction   | Reduce errors       |
| Human oversight   | Safety              |

LangGraph provides these through **graphs + cycles + state**.

---

### **2. Agent = Graph + State + Loop**

### Conceptual Model

```
┌────────┐
│ Observe│
└───┬────┘
    ↓
┌────────┐
│ Reason │
└───┬────┘
    ↓
┌────────┐
│  Act   │
└───┬────┘
    ↓
┌────────┐
│Reflect │
└───┬────┘
    ↓
  (Decision)
     ↺ Loop or Exit
```

This loop is implemented as a **cyclic LangGraph**.

---

### **3. Agent State Design**

```python
class AgentState(TypedDict):
    messages: list
    plan: str
    observation: str
    done: bool
```

The **state is the agent’s memory**.

---

### **4. Minimal Agent Implementation**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    step: int
    done: bool

def reason(state):
    return {"step": state["step"] + 1}

def act(state):
    return {}

def check(state):
    if state["step"] >= 3:
        return {"done": True}
    return {"done": False}

builder = StateGraph(State)

builder.add_node("reason", reason)
builder.add_node("act", act)
builder.add_node("check", check)

builder.set_entry_point("reason")
builder.add_edge("reason", "act")
builder.add_edge("act", "check")

builder.add_conditional_edges(
    "check",
    lambda s: END if s["done"] else "reason",
    {"reason": "reason", END: END}
)

agent = builder.compile()
agent.invoke({"step": 0, "done": False})
```

---

### **5. Core Agent Components**

| Component  | Role                  |
| ---------- | --------------------- |
| Perception | Ingests input         |
| Memory     | Stored in state       |
| Reasoning  | Decides next action   |
| Planning   | Multi-step strategy   |
| Tools      | Executes real actions |
| Reflection | Evaluates outcome     |
| Control    | Decides loop or stop  |

---

### **6. Agent Variants in LangGraph**

| Variant           | Description                 |
| ----------------- | --------------------------- |
| ReAct Agent       | Reason → Act → Observe      |
| Planner–Executor  | Plan → Execute → Replan     |
| Reflection Agent  | Draft → Critique → Revise   |
| Tool-Driven Agent | Heavy external actions      |
| Multi-Agent       | Multiple cooperating agents |
| Human-in-the-Loop | Requires approvals          |

---

### **7. Production-Grade Agent Features**

| Feature       | Implementation         |
| ------------- | ---------------------- |
| Loop control  | `recursion_limit`      |
| Safety        | Approval nodes         |
| Memory        | Persistent state store |
| Observability | Tracing + logs         |
| Recovery      | Checkpoint + replay    |
| Scalability   | Async execution        |

---

### **8. Why LangGraph Agents Are Different**

| Traditional Agents | LangGraph Agents    |
| ------------------ | ------------------- |
| Stateless          | Stateful            |
| Linear             | Cyclic              |
| Hard to debug      | Fully traceable     |
| No recovery        | Checkpoint + replay |
| Unsafe autonomy    | Controlled autonomy |

---

### **9. Mental Model**

A LangGraph agent is:

> **A continuously running decision machine whose behavior emerges from state transitions in a cyclic execution graph.**


### Demonstration

In [3]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

# -----------------------------
# 1. Define Agent State
# -----------------------------
class AgentState(TypedDict):
    step: int
    memory: list
    done: bool

# -----------------------------
# 2. Agent Nodes
# -----------------------------
def observe(state: AgentState):
    return {"memory": state["memory"] + [f"Observation at step {state['step']}"]}

def reason(state: AgentState):
    return {"memory": state["memory"] + [f"Reasoning about step {state['step']}"]}

def act(state: AgentState):
    new_step = state["step"] + 1
    return {
        "step": new_step,
        "memory": state["memory"] + [f"Action executed → new step {new_step}"]
    }

def reflect(state: AgentState):
    if state["step"] >= 3:
        return {
            "done": True,
            "memory": state["memory"] + ["Goal reached. Stopping."]
        }
    else:
        return {"memory": state["memory"] + ["Not done yet. Continue loop."]}

# -----------------------------
# 3. Build Cyclic Agent Graph
# -----------------------------
builder = StateGraph(AgentState)

builder.add_node("observe", observe)
builder.add_node("reason", reason)
builder.add_node("act", act)
builder.add_node("reflect", reflect)

builder.set_entry_point("observe")

builder.add_edge("observe", "reason")
builder.add_edge("reason", "act")
builder.add_edge("act", "reflect")

builder.add_conditional_edges(
    "reflect",
    lambda s: END if s["done"] else "observe",
    {"observe": "observe", END: END}
)

agent = builder.compile()

# -----------------------------
# 4. Run the Agent
# -----------------------------
final_state = agent.invoke(
    {"step": 0, "memory": [], "done": False},
    config={"recursion_limit": 50}
)

print("=== Agent Execution Trace ===\n")
for m in final_state["memory"]:
    print(m)

print(f"\nFinal step: {final_state['step']}")
print(f"Done: {final_state['done']}")

=== Agent Execution Trace ===

Observation at step 0
Reasoning about step 0
Action executed → new step 1
Not done yet. Continue loop.
Observation at step 1
Reasoning about step 1
Action executed → new step 2
Not done yet. Continue loop.
Observation at step 2
Reasoning about step 2
Action executed → new step 3
Goal reached. Stopping.

Final step: 3
Done: True
